# tornado-实现高并发

### 在 Tornado 中两个装饰器

* tornado.web.asynchronous
    * asynchronous 装饰器是让请求变成长连接的方式，必须手动调用 self.finish() 才会响应.
    * asynchronous 装饰器不会自动调用self.finish() ，如果没有没有指定结束，该长连接会一直保持直到 pending 状态。 
    
* tornado.gen.coroutine
    * coroutine 装饰器是指定改请求为协程模式，说明白点就是能使用 yield 配合 Tornado 编写异步程序。
    * Tronado 为协程实现了一套自己的协议，不能使用 Python 普通的生成器。Tornado 提供了多种的异步编写形式：回调、Future、协程等
    * gen.coroutine 在 Tornado 3.1 后会自动调用 self.finish() 结束请求，可以不使用 asynchronous 装饰器。

In [3]:
import os, sys
import tornado
from tornado import web
from tornado.ioloop import IOLoop
from tornado.httpserver import HTTPServer
from tornado.options import define, options

WEB_SETTINGS = {
        'autoreload': False,
        'xsrf_cookies': False,
        'cookie_secret': 'you_never_known,dummy!',
        'error': False,
        'login_url': '/api/login',
        'static_path': 'static',
        'template_path':  "templates"
    }



class MainHandler(tornado.web.RequestHandler):
    @tornado.web.asynchronous
    def get(self):
        # bad 
        self.write("Hello, world")
        self.finish()
        

        
urlhandlers = [
    (r'/main', MainHandler)
]


    
    
try:   
    app = web.Application(handlers=urlhandlers, **WEB_SETTINGS)
    myport = 8899
    print("Starting tornado web server on http://127.0.0.1:{}".format(myport))

    if sys.platform == 'win32':
        app.listen(myport)
    else:
        server = HTTPServer(app)
        server.bind(myport)
        server.start(0)

    IOLoop.current().start()     
except Exception as e:
    print("异常: {}".format(e))
    IOLoop.current().close()
    

Starting tornado web server on http://127.0.0.1:8899
异常: [WinError 10048] 通常每个套接字地址(协议/网络地址/端口)只允许使用一次。


RuntimeError: Cannot close a running event loop

使用 coroutine 方式有个**很明显是缺点**就是严重依赖第三方库的实现，如果库本身不支持 Tornado 的异步操作再怎么使用协程也是白搭依然会是阻塞的，

gen.coroutine 在 Tornado 3.1 后会自动调用 self.finish() 结束请求，可以不使用 asynchronous 装饰器。

所以这种实现异步非阻塞的方式需要依赖大量的基于 Tornado 协议的异步库，使用上比较局限，好在还是有一些可以用的异步库